In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.plotly as py
import plotly.graph_objs as go

#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)

#import plotly
#plotly.tools.set_credentials_file(username='sayasong23333', api_key='8XWuK3gwZwUj1NALsq7o')

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [28]:
df = pd.read_csv('train.csv', chunksize=100)
df = df.next()

In [31]:
train = pd.read_csv('train.csv')
train.shape

(159571, 8)

In [32]:
test = pd.read_csv('test.csv')
test.shape

(153164, 2)

### preprocessing

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.cross_validation import KFold

from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ElasticNet

/home/shj16110/anaconda2/envs/new2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [9]:
class LengthVectorizer:
    
    VEC_LEN = np.vectorize(len)

    def transform(self, data):
        return self.VEC_LEN(data).astype(float)

    def fit(self, *_):
        return self
    
    
class TextExtractor:
    def __init__(self, column):
        self.column = column

    def transform(self, data):
        return np.asarray(data[self.column]).astype(str)

    def fit(self, *_):
        return self    
    
class ArrayUpDimension:
    def transform(self, data):
        return data.reshape((-1, 1))

    def fit(self, *_):
        return self    

In [10]:
from sklearn.base import TransformerMixin
from time import time
class ModelTransformer(TransformerMixin):

    def __init__(self, model):
        self.model = model

    def fit(self, *args, **kwargs):
        start = time()
        self.model.fit(*args, **kwargs)
        print(str(self.model)+'\n used: {:.2f} s\n'.format(time()-start))
#         print (time()-start)
        return self

    def transform(self, X, **transform_params):
        return pd.DataFrame(self.model.predict(X))

In [36]:
pipeline1 = Pipeline([
    ('features', FeatureUnion([
        ('len', Pipeline([
#             ('desc_extractor', TextExtractor('comment_text')),
            ('length', LengthVectorizer()),
            ('updim_array', ArrayUpDimension()),
            ('scaler', StandardScaler()),

        ])),
        ('tf-idf', Pipeline([
#             ('desc_extractor', TextExtractor('comment_text')),
            ('count', CountVectorizer()),
            ('tf-idf', TfidfTransformer()),
        ]))
    ])),
])

In [48]:
def makePipe():
     
    pipeline2 =  Pipeline([
        ('estimators', FeatureUnion([
#             ('gbr', ModelTransformer(GradientBoostingRegressor())),
#             ('dtr', ModelTransformer(DecisionTreeRegressor())),
#             ('etr', ModelTransformer(ExtraTreesRegressor(n_jobs=-1))),
#             ('rfr', ModelTransformer(RandomForestRegressor(n_jobs=-1))),
            ('par', ModelTransformer(PassiveAggressiveRegressor())),
            ('en', ModelTransformer(ElasticNet())),
        ])),
        ('final', LogisticRegression(n_jobs=-1))
    ])
    return pipeline2

In [33]:
X_train = train['comment_text']
X_test = test['comment_text']

In [25]:
%%time
train_features = pipeline1.fit_transform(X_train)
print(type(train_features), train_features.shape)

(<class 'scipy.sparse.csr.csr_matrix'>, (159571, 189769))
CPU times: user 9.86 s, sys: 190 ms, total: 10.1 s
Wall time: 10 s


In [35]:
%%time
test_features = pipeline1.fit_transform(X_test)
print(type(test_features), test_features.shape)

(<class 'scipy.sparse.csr.csr_matrix'>, (153164, 246856))
CPU times: user 8.94 s, sys: 142 ms, total: 9.08 s
Wall time: 9.06 s


In [30]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


### pipeline each label

In [46]:
for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    y = train[label]
    model = makePipe()
    model.fit(train_features, y)
    test[label] = model.predict_proba(test)[:, 1]
    
test.drop('comment_text', axis=1, inplace=True)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)
 used: 249.61 s

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
 used: 375.50 s

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_

ValueError: could not convert string to float: " 
 :::Stop already. Your bullshit is not welcome here. I'm no fool, and if you think that kind of explination is enough, well pity you.    "

In [47]:
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...
